In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#
# **************************************************************************************************************************************************
#		Libraries
# **************************************************************************************************************************************************
#

import datetime
import numpy as np
import os
import pandas as pd
from keras import applications
from keras import optimizers
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.preprocessing.image import image, ImageDataGenerator
np.random.seed(1337)

In [0]:
#
# **************************************************************************************************************************************************
#		Global Variables
# **************************************************************************************************************************************************
#

# location of the test image
imagePath = '/content/drive/My Drive/FYP/Project/data/test/Benign/2_15.jpg'

# saved filename of model weights
weights_filename = '/content/drive/My Drive/FYP/Project/model_weights.h5'

# saved filename of result csv
results_filename = '/content/drive/My Drive/FYP/Project/results.csv'

# directories of dataset
test_data_dir = '/content/drive/My Drive/FYP/Project/data/test'

# dimensions of the architecture
img_width, img_height = 224, 224

# number of channels for the architecture
channels = 3

# learning rate
learning_rate = 1e-4

# class mode
class_mode = 'categorical'

# total number of classes
total_classes = 2

In [0]:
#
# **************************************************************************************************************************************************
#	  Main Code
# **************************************************************************************************************************************************
#

model = applications.VGG16(include_top = False, weights = 'imagenet', classes = total_classes, input_shape = (img_width, img_height, channels))
flat = Flatten()(model.layers[-1].output)
output = Dense(total_classes, activation = "softmax")(flat)
loaded_model = Model(inputs = model.inputs, outputs = output)
loaded_model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.Adam(lr = learning_rate), metrics = ['acc'])

loaded_model.load_weights(weights_filename, by_name = True)

In [0]:
# predict single test image
def test_image(path, output):
    test_image = image.load_img(path, target_size = (224, 224))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = loaded_model.predict(test_image)

    confidence = ["{:.10f}".format(r) for r in result[0]]
    if output: print("Confidence: ", confidence)

    if (result[0][0] == result[0][1]):
        if output: print("Thyroid Type: Unknown. Unable to classify this image.")
        return "Unknown", confidence
    elif (np.argmax(result, axis = 1) == 0):
        if output: print("Thyroid Type: Benign")
        return "Benign", confidence
    else:
        if output: print("Thyroid Type: Malignant")
        return "Malignant", confidence

In [41]:
# predict one image
test_image(imagePath, True)

Confidence:  ['0.0000000212', '1.0000000000']
Thyroid Type: Malignant


('Malignant', ['0.0000000212', '1.0000000000'])

In [42]:
# predict all test images
images = []
predictions = []
confidences = []

print("Predicting all test images..")
start = datetime.datetime.now()

count = 0
for (dirpath, dirnames, filenames) in os.walk(test_data_dir):
    for file in filenames:
        result, confidence = test_image(os.path.join(dirpath, file), False)
        images.append(file)
        predictions.append(result)
        confidences.append(confidence)
        count += 1

print("Writing " + str(count) + " results to CSV..")
results = pd.DataFrame({"Filename": images,
                      "Prediction": predictions,
                      "Confidence": confidences})
results.to_csv(results_filename, index = False)

end = datetime.datetime.now()
elapsed = end - start
print("Prediction completed after " + str(elapsed) + ".")

Predicting all test images..
Writing 480 results to CSV..
Prediction completed after 0:00:05.839860.
